In [1]:
import cv2
import os
import numpy as np
# import bodydetector as bt
# import find_bibs as bf
# from bib import Bib
# import BibTaggerResult as bibTaggerResult

# from swt import SWTScrubber
# import ocr
import re


In [2]:
class BibTaggerResult(object):
    def __init__(self):
        self.faces = 0
        self.bibs = 0
        self.swt = 0
        self.bib_numbers = []

    def __str__(self):
        return "Result: {0} faces, {1} bibs, {2} SWT, {3} bib numbers: ({4})".format(self.faces, self.bibs, self.swt,
                                                                                     len(self.bib_numbers),
                                                                                     self.bib_numbers)


In [3]:
### BodyDetector

def getbodyboxes(image):
    # in: numpy image
    # out: list [(x,y,width,height)]

    faces = findfaces(image)

    bodyrectangles = findbodies(image, faces)

    return bodyrectangles


def findfaces(image):
    cascPath = os.path.join("haarcascades", "haarcascade_frontalface_default.xml")

    # cascPath = os.path.join(haarcascadeFolder, "haarcascade_upperbody.xml")
    # cascPath = os.path.join(haarcascadeFolder, "haarcascade_fullbody.xml")
    # cascPath = os.path.join(haarcascadeFolder, "haarcascade_russian_plate_number.xml")

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    height, width, depth = image.shape
    scale = 1
    if width > 1024:
        scale = 1024.0 / width
        image = cv2.resize(image, None, fx=scale, fy=scale)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.05,
        minNeighbors=5,
        minSize=(30, 30),
    )

    return [scale_rect(face, 1 / scale) for face in faces]


def scale_rect(rect, scale):
    return [int(value * scale) for value in rect]


def findbodies(image, faces):
    bodies = np.zeros_like(faces)
    bodiesindex = 0

    # for each face, draw a body
    for (x, y, facewidth, faceheight) in faces:
        # 3*faceheight, 7/3 * facewidth, .5*faceheight below the face.
        bodyheight = 3 * faceheight
        bodywidth = 7 / 3 * facewidth
        y_body = y + faceheight + .5 * faceheight
        x_body = x + .5 * facewidth - .5 * bodywidth

        bodies[bodiesindex] = (x_body, y_body, bodywidth, bodyheight)
        bodiesindex = bodiesindex + 1

        # cv2.rectangle(image, (x_body, y_body), (x_body+bodywidth, y_body+bodyheight), (0, 255, 0), 2)

    return bodies


In [4]:
#### Bibtragger
class BibTaggerResult(object):
    def __init__(self):
        self.faces = 0
        self.bibs = 0
        self.swt = 0
        self.bib_numbers = []

    def __str__(self):
        return "Result: {0} faces, {1} bibs, {2} SWT, {3} bib numbers: ({4})".format(self.faces, self.bibs, self.swt,
                                                                                     len(self.bib_numbers),
                                                                                     self.bib_numbers)


#%%

def drawboxes(image, boxes, color=(0, 255, 0)):
    # Draw a rectangle around the faces
    for (x, y, w, h) in boxes:
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

def getSubImage(image, rectangle):
    # in: image, rectangle(x,y,width,height)
    # out: subimage

    (x, y, w, h) = rectangle

    return image[y:y + h, x:x + w, :]

#%%

def findBibs(image, outdir):
    # prep out dir
    writefiles = True
    if (outdir == None):
        writefiles = False
    else:
        if not os.path.exists(outdir):
            os.makedirs(outdir)

    bodyboxes = getbodyboxes(image)

    print(
    "Found {} bodies!".format(len(bodyboxes)))

    # draw bodyboxes on image, and write out
    imagecopy = np.copy(image)
    drawboxes(imagecopy, bodyboxes)
    if (writefiles):
        cv2.imwrite(os.path.join(outdir, "0_0bodyboxes.jpg"), imagecopy)

    # print(bodyboxes)
    # import pdb; pdb.set_trace()
    # Creates a list of Bibs
    bibs = [Bib(image, bodybox) for bodybox in bodyboxes]

    # Write out subimages, with bibs outlined
    if (writefiles):
        for i, bib in enumerate(bibs):
            cv2.imwrite(os.path.join(outdir, "{}_1subimage.jpg".format(i)), bib.body_image())
            cv2.imwrite(os.path.join(outdir, "{}_1subimage_withbib.jpg".format(i)), bib.body_image_with_bib())

    swtsuccesses = 0
    for i, bib in enumerate(bibs):

        SWTbib = None
        try:
            bibimage = bib.smallest_subimage_containing_bib()

            height, width, depth = bibimage.shape
            best_width = 256.0
            scale = best_width / width

            bibimage = cv2.resize(bibimage, None, fx=scale, fy=scale)
            bib.number = run_swt_and_ocr(bibimage, i, "normal", writefiles, outdir)

            if not bib.has_bib_number():
                bibimage_inverted = (255 - bibimage)
                bib.number = run_swt_and_ocr(bibimage_inverted, i, "inverted", writefiles, outdir)

            if not bib.has_bib_number():
                bibimage = cv2.cvtColor(bibimage_inverted, cv2.COLOR_BGR2GRAY);
                bibimage = cv2.GaussianBlur(bibimage, (5, 5), 0)
                bibimage = cv2.equalizeHist(bibimage)
                ret, bibimage = cv2.threshold(bibimage, 75, 255, cv2.THRESH_BINARY);
                bibimage = cv2.cvtColor(bibimage, cv2.COLOR_GRAY2BGR);
                bibimage = cv2.fastNlMeansDenoisingColored(bibimage, None, 10, 10, 7, 21)
                bib.number = run_swt_and_ocr(bibimage, i, "inverted_and_normalized", writefiles, outdir)

            # found some words in this image
            if (SWTbib is not None):
                swtsuccesses += 1

        except Bib:
            print ("SWT failed")

    result = BibTaggerResult()
    result.faces = len(bodyboxes)
    result.bibs = sum(1 for bib in bibs if bib.bib_found)
    result.swt = swtsuccesses
    result.bib_numbers = [bib.number for bib in bibs if bib.has_bib_number()]

    print (result)

    return result

In [5]:
import numpy as np

class Bib(object):
    def __init__(self, image, bodybox):
        self.image = image
        self.bodybox = bodybox
#W        self.corners = bf.find_bib(self.body_image()) 
#W         x, y, w, h = cv2.boundingRect(self.corners)
#W        self.bib_found = (x != 0 and y != 0 and w != 1 and h != 1)
        self.number = None

    def has_bib_number(self):
        return self.number != None and self.number != ''

    def body_image(self):
        return getSubImage(self.image, self.bodybox)

    def body_image_with_bib(self):
        img = np.copy(self.body_image())
        cv2.drawContours(img, [self.corners], -1, (0, 0, 255), 2)
        return img

    def corners_relative_to_main_image(self):
        x_delta = self.bodybox[0]
        y_delta = self.bodybox[1]
        return np.array([[(pt[0][0] + x_delta, pt[0][1] + y_delta)] for pt in self.corners])

    def smallest_subimage_containing_bib(self):
        if not self.bib_found:
            return self.body_image()

        return getSubImage(self.body_image(), cv2.boundingRect(self.corners))




In [6]:
def drawboxes(image, boxes, color=(0, 255, 0)):
    # Draw a rectangle around the faces
    for (x, y, w, h) in boxes:
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

In [7]:
def findBibs(image, outdir):
    # prep out dir
    writefiles = True
    if (outdir == None):
        writefiles = False
    else:
        if not os.path.exists(outdir):
            os.makedirs(outdir)

    bodyboxes = getbodyboxes(image)

    print(
    "Found {} bodies!".format(len(bodyboxes)))

    # draw bodyboxes on image, and write out
    imagecopy = np.copy(image)
    drawboxes(imagecopy, bodyboxes)
    if (writefiles):
        cv2.imwrite(os.path.join(outdir, "0_0bodyboxes.jpg"), imagecopy)

    # print(bodyboxes)
    # import pdb; pdb.set_trace()
    # Creates a list of Bibs
    bibs = [Bib(image, bodybox) for bodybox in bodyboxes]

    # Write out subimages, with bibs outlined
    if (writefiles):
        for i, bib in enumerate(bibs):
            cv2.imwrite(os.path.join(outdir, "{}_1subimage.jpg".format(i)), bib.body_image())
            cv2.imwrite(os.path.join(outdir, "{}_1subimage_withbib.jpg".format(i)), bib.body_image_with_bib())

    swtsuccesses = 0
    for i, bib in enumerate(bibs):

        SWTbib = None
        try:
            bibimage = bib.smallest_subimage_containing_bib()

            height, width, depth = bibimage.shape
            best_width = 256.0
            scale = best_width / width

            bibimage = cv2.resize(bibimage, None, fx=scale, fy=scale)
            bib.number = run_swt_and_ocr(bibimage, i, "normal", writefiles, outdir)

            if not bib.has_bib_number():
                bibimage_inverted = (255 - bibimage)
                bib.number = run_swt_and_ocr(bibimage_inverted, i, "inverted", writefiles, outdir)

            if not bib.has_bib_number():
                bibimage = cv2.cvtColor(bibimage_inverted, cv2.COLOR_BGR2GRAY);
                bibimage = cv2.GaussianBlur(bibimage, (5, 5), 0)
                bibimage = cv2.equalizeHist(bibimage)
                ret, bibimage = cv2.threshold(bibimage, 75, 255, cv2.THRESH_BINARY);
                bibimage = cv2.cvtColor(bibimage, cv2.COLOR_GRAY2BGR);
                bibimage = cv2.fastNlMeansDenoisingColored(bibimage, None, 10, 10, 7, 21)
                bib.number = run_swt_and_ocr(bibimage, i, "inverted_and_normalized", writefiles, outdir)

            # found some words in this image
            if (SWTbib is not None):
                swtsuccesses += 1

        except Bib:
            print ("SWT failed")

    result = BibTaggerResult()
    result.faces = len(bodyboxes)
    result.bibs = sum(1 for bib in bibs if bib.bib_found)
    result.swt = swtsuccesses
    result.bib_numbers = [bib.number for bib in bibs if bib.has_bib_number()]

    print (result)

    return result


In [8]:
img = cv2.imread('./photos/Frosty5k/1.jpg')
findBibs(img,"./photos-out")

Found 1 bodies!


AttributeError: 'Bib' object has no attribute 'corners'